In [1]:
import cv2
import numpy as np
import os

In [2]:
def distance(v1, v2):
    return np.sqrt(((v1 - v2) ** 2).sum())

In [3]:
def knn(train, test, k = 5):
    dist = []
    m = train.shape[0]
    for i in range(m):
        ix = train[:, :-1]
        iy = train[:, -1]
        d = distance(test, ix)
        dist.append([d, iy])

    dist = np.array(dist, dtype=object)
    dk = sorted(dist, key = lambda x:x[0])[:k]
    dk = np.array(dk, dtype=object)
    labels = np.array([int(item[1][0]) if isinstance(item[1], np.ndarray) else int(item[1]) for item in dk], dtype=int)
    output = np.unique(labels, return_counts = True)
    index = np.argmax(output[1])
    return output[0][index]
        

In [4]:
dataset_path = './Data_FC/'
class_id = 0
face_data = []
names = {}
labels = []
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

print(labels)
face_dataset = np.concatenate(face_data, axis = 0)
face_labels = np.concatenate(labels, axis = 0).reshape((-1, 1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset, face_labels), axis = 1)
print(trainset.shape)

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])]
(55, 30000)
(55, 1)
(55, 30001)


In [ ]:
cap = cv2.VideoCapture(0)
face_cascade =  cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    if(len(faces) == 0):
        continue

    for face in faces:
        x,y,w,h = face
        offset = 10
        face_section = frame[y-offset : y+h+offset, x-offset: x+w+offset]
        face_section = cv2.resize(face_section, (100,100))

        out = knn(trainset, face_section.flatten())
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,255), 2)

        cv2.imshow("Faces", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break